# COMED

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import pymssql

from icap.database.icapdatabase import ICapDatabase

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [5]:
from icap.comed.comed import COMEDInterval

In [6]:
ci = COMEDInterval(conn)

In [7]:
ci.compute_icap();

In [3]:
from icap.results.results import Results
from icap.comed.comed import COMEDRecipe

COMEDRecipe(conn, Results).run_all().analyze_comparison()

Passing                          False  True  NULL  All
MeterType RateClass Strata Year                        
INT       C28       C28    2016      0    45    13   58
                           2017      0     0    35   35
          C29       C29    2016      1    42    38   81
                           2017      0     0    82   82
          C30       C30    2016      0     4    16   20
                           2017      0     0    22   22
          C31       C31    2016      0     1     4    5
                           2017      0     0     8    8
          C48       C48    2016      0     1     0    1
                           2017      0     0     1    1
All                                  1    93   219  313

In [3]:
# Initialize excel writer
# Use the writer df.to_excel(writer, 'sheet_name')
writer = pd.ExcelWriter('comed.xlsx')

## Constants from Test Case

In [8]:
# Weather Normalized Peak Load
# Taken from the test case
COMED_NPL = 20900000;
COMED_CUST_DELTA = 1121264;

## Comed CP Avg Peak Load

In [7]:
'''
def get_comed_cp_avg_peak_load(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Cast(CPYearID - 1 as int) as Year,
            AVG(ZonalLoad) * 1000 as AvgCPZonalLoad
        from [COMED_CoincidentPeak]
        group by
            Cast(CPYearID - 1 as int)
    """
    return pd.read_sql(query, conn).set_index('Year')

get_comed_cp_avg_peak_load(conn).to_excel(writer, 'comed_cp_avg_peakload')'''

'\ndef get_comed_cp_avg_peak_load(conn: pymssql.Connection) -> pd.DataFrame:\n    query = """\n        select\n            Cast(CPYearID - 1 as int) as Year,\n            AVG(ZonalLoad) * 1000 as AvgCPZonalLoad\n        from [COMED_CoincidentPeak]\n        group by\n            Cast(CPYearID - 1 as int)\n    """\n    return pd.read_sql(query, conn).set_index(\'Year\')\n\nget_comed_cp_avg_peak_load(conn).to_excel(writer, \'comed_cp_avg_peakload\')'

In [10]:
def get_comed_cp_avg_peak_load(conn: pymssql.Connection)-> pd.DataFrame:
    query = """
        select 
            Cast(CPYearId-1 as int) Year,
            Avg(PJMZonalLoad) * 1000 as AvgCPZonalLoad
        from [CoincidentPeak]
        where UtilityId = 'COMED'
        group by
            Cast(CpYearId-1 as int)
    """
    return pd.read_sql(query, conn).set_index('Year')

In [11]:
comed_cp_avg_peak_load = get_comed_cp_avg_peak_load(conn)
comed_cp_avg_peak_load['Step4Diff'] = COMED_NPL - comed_cp_avg_peak_load['AvgCPZonalLoad'] 

In [12]:
# confirm step 4 difference
comed_cp_avg_peak_load.equals(ci.cp_avg_peak_load)

True

## PJM CP Usage

In [13]:
def get_pjm_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for PJM coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select distinct
            h.PremiseId,
            iif(RTrim(p.RateClass)  is null, 'MILES', RTrim(p.RateClass)) as RateClass,
            RTrim(p.DSC) as DSC,
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDatePJM,
            cp.HourEnding as CPHourEnding,
            --cp.HourEnding-1 as ADJCPHourEndingPJM, 
            h.Usage as UsagePJM
        from [COMED_Premise] p
        inner join [HourlyUsage] h on
            p.PremiseId = h.PremiseId
        inner join [CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding 
        where
            h.UtilityId = 'COMED'
            {prem}
        order by
            h.PremiseId, RateClass, DSC, cp.CPDate
        """
    if premise is not None:
        pjm_cp_query = query.format(prem="and h.PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="", year="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    grp = df.groupby(['PremiseId', 'Year'])['UsagePJM'].agg(
        {'CountPJM': len, 'MeanPJM': np.mean})
    grp.reset_index(inplace=True)
    
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountPJM'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanPJM', np.nan)
    
    
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year'])

In [12]:
get_pjm_cp_usage(conn).to_excel(writer, 'pjm_cp_usage')

In [13]:
df = get_pjm_cp_usage(conn)

In [14]:
df[df.PremiseId == '1199042022']

,PremiseId,RateClass,DSC,Year,CPDatePJM,CPHourEnding,UsagePJM,MeanPJM,CountPJM
1661,1199042022,MILES,C31,2016,2016-07-25,15.0,740.60,613.692,5.0
1662,1199042022,MILES,C31,2016,2016-07-27,16.0,476.56,613.692,5.0
1663,1199042022,MILES,C31,2016,2016-08-10,16.0,619.22,613.692,5.0
1664,1199042022,MILES,C31,2016,2016-08-11,15.0,589.52,613.692,5.0
1665,1199042022,MILES,C31,2016,2016-08-12,15.0,642.56,613.692,5.0


## COMED CP Usage

In [15]:
def get_comed_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for COMED coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select distinct
            h.PremiseId,
            iif(RTrim(p.RateClass)  is null, 'MILES', RTrim(p.RateClass)) as RateClass,
            RTrim(p.DSC) as DSC,
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDateCOMED, 
            cp.HourEnding as CPHourEndingCOMED, 
            h.Usage as UsageCOMED,
            ZonalLoad
        from [HourlyUsage] h
        inner join [COMED_CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding
        inner join [COMED_Premise] p on
            p.PremiseId = h.PremiseId
        where
            h.UtilityId = 'COMED'
            {prem}
        order by
            h.PremiseId, RateClass, DSC, cp.CPDate
        """
    # format query for single premise
    if premise is not None:
        pjm_cp_query = query.format(prem="and h.PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    #df.replace(to_replace=None, value="Miles", inplace=True)
    
    grp = df.groupby(['PremiseId', 'Year', 'RateClass'])['UsageCOMED'].agg(
        {'CountCOMED': len, 'MeanCOMED': np.mean})
    grp.reset_index(inplace=True)
        
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountCOMED'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanCOMED', np.nan)
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year', 'RateClass'])

In [16]:
get_comed_cp_usage(conn).to_excel(writer, 'comed_cp_usage')

In [17]:
get_comed_cp_usage(conn, premise='1199042022')

,PremiseId,RateClass,DSC,Year,CPDateCOMED,CPHourEndingCOMED,UsageCOMED,ZonalLoad,CountCOMED,MeanCOMED
0,1199042022,MILES,C31,2016,2016-07-11,17.0,587.62,20268.179688,5.0,570.464
1,1199042022,MILES,C31,2016,2016-07-21,18.0,517.93,20296.000000,5.0,570.464
2,1199042022,MILES,C31,2016,2016-08-04,17.0,557.17,20500.960938,5.0,570.464
3,1199042022,MILES,C31,2016,2016-08-11,15.0,589.52,21174.580078,5.0,570.464
4,1199042022,MILES,C31,2016,2016-09-06,17.0,600.08,20949.191406,5.0,570.464


## Load Drop Estimates

In [18]:
def get_comed_load_drop_estimates(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Cast(CPYearID -1 as INT) as Year,
            ParameterId,
            (1.0 + ParameterValue/100.0) as LoadDrop
        from [SystemLoad]
        where
            UtilityId = 'COMED' --and
            --ParameterId = 'UFT'
    """
    
    # return pd.read_sql(query, conn).set_index('Year')
    df = pd.read_sql(query, conn)
    return pd.pivot_table(df, index='Year', columns='ParameterId', values='LoadDrop')
get_comed_load_drop_estimates(conn).to_excel(writer, 'load_drop_estimates')

## Utility Factors

In [19]:
def get_comed_utility_factors(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Year(StartDate) as Year,
            RTrim(RateClass) as DSC,
            ParameterId, ParameterValue
        from [UtilityParameterValue]
        where
            UtilityId = 'COMED'
            
    """ 
    
    df = pd.read_sql(query, conn)
    
    piv = pd.pivot_table(df, index=['Year', 'DSC'], columns='ParameterId', values='ParameterValue')
    
    return piv.reset_index(level=1)
    
get_comed_utility_factors(conn).to_excel(writer, 'comed_util_factors')

In [20]:
df = get_comed_load_drop_estimates(conn)
#df.reset_index(inplace=True)
#pd.pivot_table(df, columns='ParameterId', values='LoadDrop', index='Year')
df

ParameterId,UFC,UFT
Year,,
2015,1.022679,1.035657
2016,1.022679,1.035657


In [21]:
def compute_acustcpl(conn:pymssql.Connection, premise: str=None)->pd.DataFrame:
    mean_pjm = get_pjm_cp_usage(conn, premise=premise)[['PremiseId', 'Year', 'DSC', 'MeanPJM']] \
        .drop_duplicates() \
        .reset_index()
        
    # System and util factors index on Year
    util = get_comed_utility_factors(conn)
    sys = pd.DataFrame(get_comed_load_drop_estimates(conn)['UFC'])
    
    # Join `util` and `sys` on Year index
    df = pd.merge(util, sys, left_index=True, right_index=True).reset_index()    
   
    # Join mean usage values with utility/system factors
    df = pd.merge(mean_pjm, df, on=['Year', 'DSC']).drop('index', axis=1)
 
    # Compute the AcustPL value
    df['AcustCPL'] = df['MeanPJM'] * df['DistLossFactor'] * df['TransLossFactor'] * df['UFC'] #df['LoadDrop']
    df.set_index(['PremiseId', 'Year'], inplace=True)
    return df

In [22]:
#pd.DataFrame(get_comed_load_drop_estimates(conn)['UFC'])
compute_acustcpl(conn, premise='1199042022')

,,DSC,MeanPJM,DistLossFactor,TransLossFactor,UFC,AcustCPL
PremiseId,Year,,,,,,
1199042022,2016,C31,613.692,1.0629,1.016,1.022679,677.759958


## Compute AcustPL

In [23]:
def compute_acustpl(conn: pymssql.Connection, premise: str=None)->pd.DataFrame:
    # ComedCP unique mean usage values per year; includes np.NaN
    mean_comed = get_comed_cp_usage(conn, premise=premise)[['PremiseId','Year','DSC','MeanCOMED']] \
        .drop_duplicates() \
        .reset_index()  

    # System and util factors index on Year
    util = get_comed_utility_factors(conn)
    sys = pd.DataFrame(get_comed_load_drop_estimates(conn)['UFT'])
    
    # Join `util` and `sys` on Year index
    df = pd.merge(util, sys, left_index=True, right_index=True).reset_index()    
   
    # Join mean usage values with utility/system factors
    df = pd.merge(mean_comed, df, on=['Year', 'DSC']).drop('index', axis=1)
 
    # Compute the AcustPL value
    df['AcustPL'] = df['MeanCOMED'] * df['DistLossFactor'] * df['TransLossFactor'] * df['UFT']#df['LoadDrop']
    df.set_index(['PremiseId', 'Year'], inplace=True)
    return df

In [24]:
compute_acustpl(conn).to_excel(writer, 'ACustPL')

In [25]:
compute_acustpl(conn, premise='1199042022')

,,DSC,MeanCOMED,DistLossFactor,TransLossFactor,UFT,AcustPL
PremiseId,Year,,,,,,
1199042022,2016,C31,570.464,1.0629,1.016,1.035657,638.014127


## AcustCPL

In [26]:
# Import all usage data for utility or premise
# returns pd.DataFrame indexed on [PremiseId, Year]
pjm_cp_usage = get_pjm_cp_usage(conn) \
    .set_index(['PremiseId', 'Year'])

    
# AcustCPL
# AcustCPL = pjm_cp_usage.MeanPJM * (1.0 + SystemLoad.UFT) * UPV.[DistLossFactor, TransLossFactor]
#AcustCPL = pjm_cp_usage['MeanPJM'].drop_duplicates()
AcustCPL = compute_acustcpl(conn)['AcustCPL']

compute_acustcpl(conn).to_excel(writer, 'ACustCPL')

In [27]:
comed_cp_usage = get_comed_cp_usage(conn) \
    .set_index(['PremiseId', 'Year'])
    
AcustPL = compute_acustpl(conn)['AcustPL']

In [28]:
peak_loads = pd.merge(pd.DataFrame(AcustCPL), pd.DataFrame(AcustPL),
                        left_index=True, right_index=True) #\
                #.rename(columns={'MeanPJM': 'AcustCPL'})


#peak_loads.to_excel(writer, 'peak_loads')

In [29]:
    
df = pd.merge(peak_loads, comed_cp_avg_peak_load, left_index=True, right_index=True)

In [30]:
def step6(r: pd.Series) -> np.float32:
    if r.AcustCPL >= r.AcustPL:
        return r.AcustCPL
    
    return ((r.AcustPL - r.AcustCPL) / COMED_CUST_DELTA ) # *Step4Diff  #+ r.AcustCPL
    
#f['Step6'] = (df['AcustPL'] - df['AcustCPL']) / COMED_CUST_DELTA


def icap(r: pd.Series) -> np.float32:
    if r.AcustCPL == r.Step6:
        return r.AcustCPL
    return r.AcustCPL + r.Step7


df['Step6'] = df.apply(step6, axis=1)
df['Step7'] = df['Step6'] * df['Step4Diff']
 

    

    
#df['ICap'] = df['Step7'] + df['AcustCPL']
df['ICap'] = df.apply(icap, axis=1)
#
#df.to_excel(writer, 'icap')

In [31]:
df.xs('0443154096', level=0)

,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap
Year,,,,,,,
2015,327.790434,324.786698,18880660.0,2019340.0,327.790434,6.619203e+08,327.790434


<hr>

## Compare against historical

In [32]:

historical_query = """
    select 
        PremiseId, Cast(CPYearID-1 as INT) Year,
        CapacityTagValue as Historical
    from [CapacityTagHistorical]
    where
        UtilityId = 'COMED'
    """

historical = pd.read_sql(historical_query, conn).set_index(['PremiseId','Year'])

In [33]:
def one_kw(r: pd.Series)-> str:
    if np.abs(r['ICap'] - r['Historical']) <= 1.0:
        return 'true'
    return 'false'

def passing(r: pd.Series) -> np.int32:
    if abs(r.Variance) <= 2.0 or r['1KW'] == 'true':
        return 1
    return 0

compare = pd.merge(df, historical,
         left_index=True, right_index=True, how='left')


compare['Variance'] = (compare['Historical'] - compare['ICap']) / compare['Historical'] * 100
compare['1KW'] = compare.apply(one_kw, axis=1)

compare['Passing'] = compare.apply(passing, axis=1)

/home/miles/miniconda3/envs/justenergy/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in absolute
  from ipykernel import kernelapp as app


In [34]:
compare.ix['9590268008']

,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap,Historical,Variance,1KW,Passing
Year,,,,,,,,,,,
2015,101.761441,176.919427,18880660.0,2019340.0,0.000067,1.353557e+02,237.117189,237.1173,0.000047,true,1
2016,164.854017,113.833523,19227060.0,1672940.0,164.854017,2.757909e+08,164.854017,NaN,NaN,false,0


## Output to Excel for analysis

In [35]:
query = "select * from COMED_CoincidentPeak where CPYearID = 2016. order by CPDate"
pd.read_sql(query, conn)

,CPID,CPYearID,ISOID,CPDate,HourEnding,UtilityID,ZonalLoad
0,9,2016.0,PJM,2015-07-17,16.0,COMED,19525.496094
1,7,2016.0,PJM,2015-07-28,16.0,COMED,19765.029297
2,10,2016.0,PJM,2015-09-01,17.0,COMED,19424.433594
3,8,2016.0,PJM,2015-09-02,15.0,COMED,19715.060547
4,6,2016.0,PJM,2015-09-03,17.0,COMED,20162.302734


In [36]:
#writer = pd.ExcelWriter('comed.xlsx')
compare.reset_index(inplace=True)
compare.to_excel(writer, 'variances')

#comed_cp_avg_peak_load.to_excel(writer,'avg_peak_loads')

#AcustCPL.to_excel(writer, 'AcustCPL')
#AcustPL.to_excel(writer, 'ACustPL')
#df.to_excel(writer, 'all_data')
writer.save()

In [37]:
#compare.xs('1199042022', level=0)

In [38]:
compare.to_csv(path_or_buf='~/Projects/BusinessLab/JustEnergy/comed.csv')

# Comparing to Test Cases

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('/home/miles/Dropbox/iCAP_Project/Results/Analysis/comed.csv')

In [41]:
tc1 = '9590268008'

In [42]:
df[df['PremiseId'] == 9590268008]

,Unnamed: 0,PremiseId,Year,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,ICap,Historical,Variance,1KW,Passing
303,303,9590268008,2015,103.052814,176.919427,1.971846e+07,1.181536e+06,180.889995,103.7372,-74.373315,False,0
304,304,9590268008,2016,166.946048,113.833523,2.063778e+07,2.622176e+05,166.946048,237.1173,29.593476,False,0


In [43]:
compare[compare.PremiseId == '0443154096']


,PremiseId,Year,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap,Historical,Variance,1KW,Passing
59,0443154096,2015,327.790434,324.786698,18880660.0,2019340.0,327.790434,6.619203e+08,327.790434,327.7904,-0.000011,true,1


In [44]:
compare[compare.PremiseId == '9590268008']

,PremiseId,Year,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap,Historical,Variance,1KW,Passing
304,9590268008,2015,101.761441,176.919427,18880660.0,2019340.0,0.000067,1.353557e+02,237.117189,237.1173,0.000047,true,1
305,9590268008,2016,164.854017,113.833523,19227060.0,1672940.0,164.854017,2.757909e+08,164.854017,NaN,NaN,false,0


In [45]:
print(compare.dropna(subset=['Variance']).shape[0])

compare[((compare.Variance <= 2.0) | (compare.Historical - compare.ICap <= 1))].dropna(subset=['Variance']).sort_values(by='Variance').shape[0]

94


93

In [46]:
compare.dropna(subset=['Variance']).sort_values(by='Passing')

,PremiseId,Year,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap,Historical,Variance,1KW,Passing
123,1043656025,2015,190.586286,192.495186,18880660.0,2019340.0,1.702454e-06,3.437833e+00,194.024119,208.6073,6.990734,false,0
226,4899012067,2015,85.995648,97.412696,18880660.0,2019340.0,1.018230e-05,2.056153e+01,106.557176,106.5573,0.000116,true,1
224,4842743019,2015,80.581608,76.950505,18880660.0,2019340.0,8.058161e+01,1.627217e+08,80.581608,80.5816,-0.000010,true,1
220,4317412038,2015,77.141976,83.121441,18880660.0,2019340.0,5.332790e-06,1.076872e+01,87.910692,87.9108,0.000123,true,1
214,3972754012,2015,104.080606,108.051736,18880660.0,2019340.0,3.541654e-06,7.151804e+00,111.232410,111.2325,0.000081,true,1
212,3907015055,2015,181.200913,190.040577,18880660.0,2019340.0,7.883660e-06,1.591979e+01,197.120703,197.1209,0.000100,true,1
210,3843017008,2015,90.549457,95.713451,18880660.0,2019340.0,4.605511e-06,9.300093e+00,99.849550,99.8496,0.000050,true,1
208,3709079008,2015,186.266035,173.520894,18880660.0,2019340.0,1.862660e+02,3.761345e+08,186.266035,186.2660,-0.000019,true,1
205,3451167019,2015,12.618520,13.567424,18880660.0,2019340.0,8.462804e-07,1.708928e+00,14.327448,14.3275,0.000363,true,1
194,2787080257,2015,60.841440,67.068641,18880660.0,2019340.0,5.553733e-06,1.121488e+01,72.056315,72.0564,0.000117,true,1


In [47]:
93 / 94

0.9893617021276596